In [2]:
import pandas as pd
from pathlib import Path
import os

In [3]:
PROJECT_ROOT = Path(os.getcwd()).resolve().parents[0]

In [4]:
df = pd.read_csv("stf_cd_results.csv")

In [5]:
df.columns

Index(['Region', 'Partenaire', 'Contrat', 'Activity', 'Year', 'Month',
       'Consommation_Total', 'Consommation_ZCONHC', 'Consommation_ZCONHL',
       'Consommation_ZCONHP', 'Puissance_Facturee', 'Niveau_tension'],
      dtype='object')

In [6]:
df["Contrat"].nunique()

136

In [17]:
df = pd.read_csv("stf_cd_results.csv")

In [9]:
df["Contrat"].nunique()

146

In [18]:
df_correction = pd.read_csv("stf_cd_results_correction.csv")

In [19]:
df_correction["Contrat"].unique()

array([ 217229, 8303294, 5022460, 6457296, 6955110, 8315738, 6457387,
       8279211, 6135297, 5719910, 6434280])

In [20]:
df_safe = df.copy()

df_safe = df_safe.set_index("Contrat")
df_corr = df_correction.set_index("Contrat")

df_safe.update(df_corr)

df_safe = df_safe.reset_index()


ValueError: cannot reindex on an axis with duplicate labels

In [21]:
df_safe = df.merge(
    df_correction,
    on="Contrat",
    how="left",
    suffixes=("", "_corr")
)

for col in df_correction.columns:
    if col != "Contrat":
        df_safe[col] = df_safe[f"{col}_corr"].combine_first(df_safe[col])
        df_safe.drop(columns=f"{col}_corr", inplace=True)


In [23]:
df_safe.columns

Index(['Region', 'Partenaire', 'Contrat', 'Activity', 'Year', 'Month',
       'Consommation_Total', 'Consommation_ZCONHC', 'Consommation_ZCONHL',
       'Consommation_ZCONHP', 'Puissance_Facturee', 'Niveau_tension'],
      dtype='object')

In [27]:
df_safe[df_safe["Consommation_Total"] < 0]

,Region,Partenaire,Contrat,Activity,Year,Month,Consommation_Total,Consommation_ZCONHC,Consommation_ZCONHL,Consommation_ZCONHP,Puissance_Facturee,Niveau_tension


In [28]:
df_safe.isna().sum()

Region                 0
Partenaire             0
Contrat                0
Activity               0
Year                   0
Month                  0
Consommation_Total     0
Consommation_ZCONHC    0
Consommation_ZCONHL    0
Consommation_ZCONHP    0
Puissance_Facturee     0
Niveau_tension         0
dtype: int64

In [29]:
df_safe.to_csv("stf_cd_results_final.csv", index=False)